# Introduction to earthquake signal processing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GPGN-268/GPGN268-CORE/blob/main/assignments/earthquakes/eq_processing.ipynb)

The this notebook provides a gentle introduction to processing seismic signals of earthquakes. We will use the excellent open-source libraries [ObsPy](https://github.com/obspy/obspy), [SeisBench](https://github.com/seisbench/seisbench), and [PyOcto](https://github.com/yetinam/pyocto) for this exercise.

After completing the assignment you will know how to:

1. Pre-process earthquake time series data using filtering and instrument response removal
2. Plot seismograms, their spectra, and create spectrograms
3. Automatically make P/S picks, associated the picks into events, and roughly locate the earthquakes

Additional resources you may find helpful include:

- [The ObsPy tutorial](https://docs.obspy.org/tutorial/)
- [SeisBench example notebooks](https://github.com/seisbench/seisbench?tab=readme-ov-file#getting-started)
- [PyOcto](https://github.com/yetinam/pyocto)
- [OOP in python](https://realpython.com/python3-object-oriented-programming/)

**->Within this notebook, you need to complete two types of tasks<-**

- Code cells that start with "## Exercise" provide instructions for you to complete in code.
- Text cells which have -> followed by a bold prompt are questions for you to answer.

Before getting started, we need to install the required dependencies.

In [ ]:
#@title Install packages
%%capture

# First ensure ObsPy and Seisbench are installed. We need to restart kernel if not.
try:
    import obspy
    import seisbench
    import numpy as np
except ImportError:
    !pip install obspy
    !pip install seisbench
    !pip install cartopy
    !pip install pyocto
    # resetart kernel
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Get a client for downloading data
from obspy.clients.fdsn.client import Client

client = Client()

# Part 1: A brief introduction to ObsPy

Like many python libraries, ObsPy implements an object-oriented interface. This means it implements data structures (known as classes) which are used to create *instances*, or *objects*, which manage data and provide methods for operating on the data. If you aren't familiar with this terminology, don't worry, we will go through several examples. The [Real Python OOP Guide](https://realpython.com/python3-object-oriented-programming/) is also a great resource.

For this assignment, you will use a few ObsPy classes. The main ones are [`UTCDateTime`](https://docs.obspy.org/tutorial/code_snippets/utc_date_time.html), [`Stream`](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html), and [`Trace`](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.html).

In [ ]:
#@title Part 1 helper functions
# Helper functions for this section. Just run this cell and move on.

def get_fourier_spectrum(trace):
    """
    Return the Fourier Spectrum from the trace and corresponding frequencies.

    Parameters
    ----------
    trace
        The obspy trace to transform to Fourier domain.

    Notes
    -----
        Since the trace data is assumed to be real, the fft is symmetric
        so only the right half (possitive frequencies) are needed.
    """
    fft = np.fft.rfft(trace.data)
    # Get the temporal spacing of data.
    sample_spacing = 1 / trace.stats.sampling_rate
    freqs = np.fft.rfftfreq(len(trace.data), sample_spacing)
    return fft, freqs



## Time

For historical reasons, ObsPy uses `UTCDateTime` objects to represent [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time) times. A precise and unified time system is important for studying earthquakes. `UTCDateTime` objects can be created from several formats, including:

1. An [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) string, e.g., "2020-01-03T12:13:23"
2. A number representing seconds from [epoch time](https://en.wikipedia.org/wiki/Unix_time) (Jan 1st 1970).


In [ ]:
# Create time from an ISO 8601 string.
time_from_str = obspy.UTCDateTime("2017-09-17T12:01:02.012344")
# Create time from one billion seconds after 1970.
time_from_number = obspy.UTCDateTime(1_000_000_000)
# Get the time right now (according to your computer's clock).
time_now = obspy.UTCDateTime()

You can also create new `UTCDateTime` objects by adding or subtracting any number of seconds.

Subtracting `UTCDateTime` objects gives the difference in seconds.

In [ ]:
# 1_000.01 seconds in the future.
future_time = time_now + 1_000.01

# Get the time 100 days in the past (24 hours in a day, 3600 seconds in an hour).
past_time = time_now - 100 * 24 * 3600

# Get the number of seconds between Guy Fawkes day and Pi Day in 1999.
total_seconds = obspy.UTCDateTime("1999-11-05") - obspy.UTCDateTime("1999-03-14")

In [ ]:
## Excercise
# Create a UTCDateTime object of your birthday.

In [ ]:
## Excercise
# Calculate and print how many hours from now to your next birthday.

## Streams and Traces

The `Trace` contains waveform data (a 1D numpy array representing instrument output) and metadata (station name, sampling rate, etc.). It also has several *methods*, which operate on the data/metadata.

The `Stream` is just a container for traces. This graphic is helpful:

![](https://docs.obspy.org/pr/filter/_images/Stream_Trace.png)


In [ ]:
# Create a Stream.
stream = obspy.read()  # You can also pass a path to a data file.
# Plot all of the traces in the stream.
stream.plot();

ObsPy `Stream` instances behave like lists.

In [ ]:
# Get the first trace in a list.
trace = stream[0]
trace.plot();

In [ ]:
# Print the metadata about the trace.
print(trace.stats)

In [ ]:
# Print the station name.
station_name = trace.stats.station
print(station_name)

In [ ]:
# Get the time series data as a numpy array.
data = trace.data
print(data)

In [ ]:
## Excercise
# Using the starttime and endtime in the stats, calcuate the duration of the
# trace in seconds. Then print it.

In [ ]:
## Excercise
# Without hardcoding any values, print how many traces are in the stream.
# Hint: remember streams behave like lists.

### Trimming
Traces and streams can be trimmed using the `slice` method. This returns a new trace object.

There is also a `trim` method, but this modifies the trace/stream in place, so `slice` is safer and should be preferred.

In [ ]:
# Trim 4 second from the start and 15 seconds from the end of the trace.
new_starttime = trace.stats.starttime + 4
new_endtime = trace.stats.endtime - 15
trimmed = trace.slice(starttime=new_starttime, endtime=new_endtime)
trimmed.plot();

In [ ]:
## Excercise
# Create and plot a trace which zooms into the first 5 seconds of the event.

### Detrending

Detrending is used to remove a trend from data. Both `Stream` and `Trace` have a [detrend](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.detrend.html) method. This modifies the data in place so we first make a copy in case we want to use the original data again later.

In [ ]:
# Create a trace with a linearly increasing trend.
trace_with_trend = trace.copy()
trend = np.linspace(0, trace.data.max()*2, len(trace.data))
trace_with_trend.data += trend

print("Plotting trace with trend")
trace_with_trend.plot();

In [ ]:
# Remove the trend.
detrended_trace = trace_with_trend.copy().detrend()
detrended_trace.plot();

In [ ]:
## Excercise
# Use a "constant" type of detrend on the trace_with_trend and plot.
# Hint: Use the link above to see the detrend docs.

-> *Why do the two detrended traces look different?*


## Filtering
The [filter method](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.filter.html) of both streams and traces is very useful. It allows you to selectively attenuate certain frequencies in the data.

In the example below, notice how the low-frequency background noise is removed.

In [ ]:
# Attenuate frequencies outside 1 to 10 Hz.
# The .copy() is again used to avoid changing the original trace.
tr_bp_filtered = trace.copy().filter("bandpass", freqmin=1, freqmax=10)
tr_bp_filtered.plot();

In [ ]:
## Excercise
# Apply a 10 Hz highpass filter to the trace and plot it.
# Hint: Don't forget .copy() before filtering and use the link above to see
# the filter docs.

In [ ]:
## Excercise
# Plot the amplitude spectra of the original trace and the band-passed trace on
# the same plot. Label each line, the axes, and show the legend.
# Hint: use the helper function `get_fourier_spectrum` defined above.

**Does this make sense given the filtering we performed?**

Yes, the spectra are largely the same within the 1 to 10 Hz range, then the fitlered spectrum is nearly 0 outside of this range.


## Spectrograms

A [Spectrogram](https://en.wikipedia.org/wiki/Spectrogram) is a type of plot that captures captures temporal variations in frequency. It is created from a series of short (often overlapping) time windows which are transformed to the Fourier domain. See ObsPy's [spectrogram docs](https://docs.obspy.org/packages/autogen/obspy.imaging.spectrogram.spectrogram.html#obspy.imaging.spectrogram.spectrogram) for more details.

In [ ]:
trace.spectrogram()

**-> Looking at the spectrogram, does the frequency increase or decrease as time progresses in seismic signal?**

It decreases.

## Instrument response

The signal recorded by a seismic sensor, $s(t)$ is not the actual ground motion $u(t)$ but the ground motion convolved with an instrument response $r(t)$. This can be represented in the time domain with the convolution operator ($\circledast$):

$$
s(t) = u(t) \circledast r(t)  
$$

or in the frequency domain with spectral multiplication:

$$
S(f) = U(f) R(f)
$$


To get the ground motion from the output of a seismic instrument, which we might want to estimate magnitudes, we need to remove the response. In the simplest form this involves dividing the recorded spectrum by the instrument response spectrum. However, oddities in instrument response file formats and the need to avoid dividing small numbers makes this a bit more complicated. Luckily, ObsPy helps with those details.

Response information can either be attached to a trace or contained in an `Inventory`. In the case of the default trace/stream, the responses are already attached as `stats.response`.

To see how this works, we will download waveforms and station data associated with a [large mine disaster](https://en.wikipedia.org/wiki/Crandall_Canyon_Mine).



In [ ]:
# Download waveforms
time = obspy.UTCDateTime("2007-08-06T08:48:40")
cc_trace = client.get_waveforms(
    network='US',
    station='DUG',
    starttime=time + 25,
    endtime=time+120,
    location='*',
    channel='BHZ',
)[0]

In [ ]:
cc_trace.plot();

In [ ]:
# Download the station response
inv = client.get_stations(
    starttime=time,
    endtime=time+120,
    network="US",
    station="DUG",
    location="*",
    channel="BHZ",
    level="response",
)

In [ ]:
cc_trace_no_response = cc_trace.copy().remove_response(inventory=inv)

In [ ]:
cc_trace_no_response.plot();

In [ ]:
## Exercise
# Get the normalized (by dividing by max value) amplitude spectra of the
# downloaded trace and the one with the response removed.
# Plot both responses.
# hint: again, get_fourier_spectrum will be useful.

---
# Part 2: A brief introduction to SeisBench

[SeisBench](https://github.com/seisbench/seisbench) is a python library for applying machine learning in Seismology. It provides a common interface to many pre-trained deep learning models to automate earthquake processing tasks.

It really is magic... but sometimes the [dangerous kind](https://en.wikipedia.org/wiki/The_Monkey%27s_Paw) that does unexpected things.

In [ ]:
#@title Part 2 helper functions

# Utility code for this section. Simply run this cell and move on.

def to_df(list_thing):
    """
    Convert a list-like thing to a dataframe.

    Also convert columns of obspy UTCDateTime objects to numpy datetime64.

    Parameters
    ----------
    list_thing
        A collection of objects which have `__dict__`.
    """
    out = [x.__dict__ for x in list_thing]
    df = pd.DataFrame(out)
    # Convert all times columns to numpy datetime64.
    for col in df.columns:
        if 'time' in col:
            df[col] = [
                np.datetime64(str(x)[:26] )
                for x in df[col].values
            ]
    # Add "time" column which will be used by the associator. Must be
    # a float
    diffs = df['peak_time'] - df['peak_time'].min()
    df['time'] = diffs / np.timedelta64(1, 's')
    # Need to add station code.
    split = df['trace_id'].str.split(".", expand=True)
    df['network'] = split[0]
    df['station'] = split[1]
    return df


## A New Dataset

First, we download one hour of data starting 10 minutes after the main event. We need all three components (up, east, and north) for the picker.  

In [ ]:
import seisbench.models as sbm

In [ ]:
# Download waveforms, note: we need all 3 components for this picker.
time_1 = obspy.UTCDateTime("2007-08-06T08:48:40") + 10*60
time_2 = time_1 + 3600
cc_stream = client.get_waveforms(
    network='UU',
    station='SRU',
    starttime=time_1,
    endtime=time_2,
    location='*',
    channel='HH?',
)

In [ ]:
cc_stream.plot();

It is difficult to see more than 1 or 2 aftershocks, but they are there!

Let's try a highpass filter to bring them out.

In [ ]:
hp_cc_stream = cc_stream.copy().filter("highpass", freq=1)

hp_cc_stream.plot();

## Deep denoiser

Maybe we can do better? Let's try the [Deep Denoiser](https://ieeexplore.ieee.org/abstract/document/8802278) on the highpass data. We do this by creating a seisbench model and loading the trained model weights.

Then we use the `annotate` method to return a new `Stream` object with the model outputs.

In [ ]:
# Create a denoiser model from the original training weights.
denoiser = sbm.DeepDenoiser.from_pretrained("original")

In [ ]:
# Denoise the high-passed stream and plot.
denoised_st = denoiser.annotate(hp_cc_stream)
denoised_st.plot();

In [ ]:
## Excercise
# Try denoising the original stream (cc_stream) and plotting it.
# hint: don't overwrite the variable "denoised_st", we will use it later.

-> **Comparing this output with the previous, what indications are there something might be going wrong?**

## Phase Picking

Prior to sophisticated deep-learning models, phase picking was largely done manually, or high errors produced by simple models were tolerated. Now, automatic picking can be performed with near-human performance. This is a major time saver and enables the analysis of larger datasets.



 One of the most popular phase pickers is called [PhaseNet](https://academic.oup.com/gji/article/216/1/261/5129142). It is a [U-Net](https://en.wikipedia.org/wiki/U-Net) trained on California earthquakes.

 The Phasenet model's `annotate` method returns a new stream, with each trace representing the estimated probability of noise, P, and S, for each sample, respectively.

In [ ]:
# Get a phasenet picker.
pn_model = sbm.PhaseNet.from_pretrained("original")

In [ ]:
# Get probability estimates of each part of the stream being a P or S arrival.
pn_preds = pn_model.annotate(cc_stream)
pn_preds.plot(show=False)

The `classify` method parses the output and returns a list of picks. Picks are made when the probability of P or S phases exceeds some threshold.

In [ ]:
# Make pick estimates.
picks = pn_model.classify(cc_stream).picks

In [ ]:
# Convert picks to a table (pandas dataframe).
pick_df = to_df(picks)
pick_df.head()

In [ ]:
## Excercise
# Try running the same process on the denoised_st. Print how many picks it
# found.

-> **Did performing the phase picking on the denoised work better? How do you know?**

It returned more than 4x the picks, but we aren't sure if these are all real.

SeisBench has several phasenet weights (run `sbm.PhaseNet.list_pretrained()` to see them) and other models for phase picking such as `GPD` and `EQTransformer` which each have several options for weights.

In [ ]:
## Excercise
# Select another phase picker or phasenet weight and run it again on the
# denoised data. Plot the original phase net picks and the new ones.
# The peak_time should be the x axis and peak_value on the y axis.
# Hint: Use a dot, x, or other symbol. The line plot looks messy.

In [ ]:
## Excercise
# Using the same two pickers as before, get phase picks from the original
# (not denoised) data and plot.
# Hint: This should be nearly identical as the previous excercise, just
# change one variable name.

-> **Looking at both outputs, which do you trust more and why?**


# Part 3: Aftershock Processing

For the final section, we will detect and locate aftershocks from the [2020 Salt Lake City Earthquake](https://en.wikipedia.org/wiki/2020_Salt_Lake_City_earthquake#:~:text=At%207%3A09%20AM%20MDT,the%20planned%20Utah%20Inland%20Port.). This is the event we studied with our analog processing activity in class.

The following table shows basic information about the event:

|                |                     |
|----------------|---------------------|
| UTC time       | 2020-03-18T13:09:31 |
| Magnitude (Mw) | 5.7                 |
| Depth          | 11.7km              |
| Latitude       | 40.851 N            |
| Longitude      | 112.081 W           |


In [ ]:
#@title Section 3 Utilities
# Utility code for this section. Simply run this cell and move on.
from collections import Counter
from pathlib import Path

import pyocto


def load_seismic_data(path, only_3c=True):
    """
    Load the waveform and station data in a directory.

    The directory must have subfolders name "waveforms" and "stations".

    Parameters
    ----------
    path
        The directory path which contains the data.
    only_3c
        If True, only load data that has 3 components.
    """

    path = Path(path)
    # Ensure expected paths exist.
    wave_folder_path = path / "waveforms"
    sta_folder_path = path / "stations"
    assert wave_folder_path.is_dir(), f"{wave_folder_path} does not exist"
    assert sta_folder_path.is_dir(), f"{sta_folder_path} does not exist"
    # Load waveforms into stream.
    st = obspy.Stream()
    for wpath in wave_folder_path.glob("*.mseed"):
        st += obspy.read(wpath)
    # Filter stream to only include stations with 3 components.
    if only_3c:
        count = Counter([tr.stats.station for tr in st])
        has_3c = {i for i, v in count.items() if v >= 3}
        st = obspy.Stream([tr for tr in st if tr.stats.station in has_3c])
    # Load station info into inventory.
    inv = obspy.Inventory()
    stations = {tr.stats.station for tr in st}
    for sta_path in sta_folder_path.glob("*.xml"):
        sub_inv = obspy.read_inventory(sta_path)
        # Only include stations that are in stream.
        if sub_inv[0][0].code in stations:
            inv += sub_inv
    return st, inv



def inventory_to_df(inv):
    """
    Extract station names and locations to a dataframe.

    Parameters
    ----------
    inv
        The obspy Inventory object.
    """
    data = []
    for network in inv.networks:
        for station in network.stations:
            sta_info = {
                "network": network.code,
                "station": station.code,
                "latitude": station.latitude,
                "longitude": station.longitude,
                "elevation": station.elevation,
                "id": network.code + '.' + station.code + ".'",
            }
            data.append(sta_info)
    return pd.DataFrame(data)


## Download data
ObsPy's [Mass Data Downloader](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html) provides a convinient way to, well, download lots of seismic data. We will use it to get waveforms and station info about the event.

NOTE: All the data are saved in a directory called "data" which you can find in the file explorer.

In [ ]:
#@title Download aftershock data
%%capture

from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader

origin_time = obspy.UTCDateTime(2020, 3, 18, 13, 9, 31)

time_1 = origin_time + 900
time_2 = time_1 + 60 * 60

latitude = 40.851
longitude = -112.081

# Circular domain around the epicenter. This will download all data between
# 0.5 and 3 degrees distance from the epicenter.
domain = CircularDomain(latitude=latitude, longitude=longitude,
                        minradius=0.0, maxradius=2.5)

restrictions = Restrictions(
    # Get data 10 second before the event to 2 minutes after.
    starttime=time_1,
    endtime=time_2,
    # Reject gappy/missing data
    reject_channels_with_gaps=True,
    minimum_length=1.0,
    # No two stations should be closer than 10 km to each other.
    minimum_interstation_distance_in_m=10E3,
    # Just get stations from UUSS.
    network='UU',
    # Only broadband (HH or BH) channels.
    channel_priorities=["HH[ZNE]", "BH[ZNE]"],
    # location_priorities=["", "00", "10"])
)

mdl = MassDownloader(["IRIS"])
# The data will be downloaded to the ``mainshock/waveforms/`` and ``mainshock/stations/``
# folders with automatically chosen file names.
mdl.download(domain, restrictions, mseed_storage="data/waveforms",
             stationxml_storage="data/stations")

## PyOcto demo

[PyOcto](https://github.com/yetinam/pyocto) is an efficient pick associator. Essentially, it takes a basic velocity model and station locations into account to find locations and picks which are consistent with seismic events.

The following code demonstrates how to use PyOcto with SeisBench.

In [ ]:
# Load the streams and inventory.
st, inv = load_seismic_data("data")

In [ ]:
# Use the original phase net model to make picks.
pn_model = sbm.PhaseNet.from_pretrained("original")
picks = pn_model.classify(st).picks

In [ ]:
# Create the PyOcto associator.
velocity_model = pyocto.VelocityModel0D(
    p_velocity=6,
    s_velocity=3.0,
    tolerance=2.0,
)

associator = pyocto.OctoAssociator.from_area(
    lat=(39, 43),
    lon=(-114, -110),
    zlim=(0, 100),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=9,
    n_p_picks=3,
    n_s_picks=2,
    n_p_and_s_picks=1,
)

Prepare input data for the associator.

In [ ]:
# PyOcto provides a way to convert an inventory to a Pandas dataframe.
stations = associator.inventory_to_df(inv)

In [ ]:
# Then we simply feed the associator the picks and stations to it.
events, assignments = associator.associate_seisbench(picks, stations)

In [ ]:
# And use transform to add lat/lon back to events df.
associator.transform_events(events)

# Convert timestamps to datetimes.
events['time'] = pd.to_datetime(events['time'], unit='s')

In [ ]:
events

In [ ]:
## Exercise
# Above we found 5 aftershocks. Using the techniques you learned in previous
# sections,be creative, and see if you can do better. You can also play around
# with the input parameters to the associator. Then make a map plot of the
# aftershocks you found by coloring the dots according to number of picks.
# Also show the original event as a star and the stations as triangles.


-> **Comment on the apparent clustering of the aftershocks and over quality. Is there anything to indicate some may be noise?**
